### dNBR to Geotiff
Created by Amos Bennett.<br>
Last Updated 12 Jun 20. <br>
Updates: Script was pulled from DEA sandbox and loaded to NCI VDI after an error with not sourcing s2a/b_ard_granule and s2a/b_nrt_granule correctly. Because the fires being analysed occured several months ago, nrt (near real time) is no longer required and was removed. The DEA startup on the NCI VDI side was set up to be linked to module DEA version 1.8.0 due to a deprecation issue with write_geotiff().

In [38]:
import sys
import datacube
from datacube.helpers import write_geotiff
from datetime import datetime
from datetime import timedelta
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../Scripts")
import dea_datahandling
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map
from dea_bandindices import calculate_indices

In [39]:
print(datacube.__version__)

1.8.0


In [40]:
dc = datacube.Datacube(app="dNBR")

In [41]:
# Set the central latitude and longitude
central_lat = -35.5655
central_lon = 148.6547

# Canberra 148.6547665°E 35.5655761°S
# Bago 148.1578780°E 35.6453203°S
# Koziosko 148.3517111°E 36.1864717°S 

# Set the buffer to load around the central coordinates (even numbers such as 0.2, 1.0, 2.2 etc)
buffer = 0.6

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

/g/data/v10/public/modules/dea-env/20200605/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/g/data/v10/public/modules/dea-env/20200605/lib/python3.6/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/g/data/v10/public/modules/dea-env/20200605/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<a

In [42]:
import shapely
from shapely.geometry import Point

# Create a range for generating point grid.
magic_number = int(buffer*10/1)
rng = range(int(magic_number/2))

x_coord = []
y_coord = []

# Calculate x and y for point grid.
for i in rng:
    l = i/5 + 0.1
    neg_lat = central_lat - l
    neg_lon = central_lon - l
    pos_lat = central_lat + l
    pos_lon = central_lon + l
    x_coord.append(neg_lon)
    x_coord.append(pos_lon)
    y_coord.append(neg_lat)
    y_coord.append(pos_lat)
    
coords = []

# Create list of shapely points.
for x in x_coord:
    for y in y_coord:
        p = Point(x, y)
        coords.append(p)

In [43]:
# Fire event date
fire_date = '2020-02-20'

# Length of baseline period
baseline_length = '3 months'

In [44]:
# Define dates for loading data
if baseline_length == '12 months':
    time_step = timedelta(days=365)
if baseline_length == '6 months':
    time_step = timedelta(days=182.5)
if baseline_length == '3 months':
    time_step = timedelta(days=91)

# Calculate the start and end date for baseline data load
start_date_pre = datetime.strftime(((datetime.strptime(fire_date, '%Y-%m-%d'))-time_step), '%Y-%m-%d')
end_date_pre = datetime.strftime(((datetime.strptime(fire_date, '%Y-%m-%d'))-timedelta(days=25)), '%Y-%m-%d')

# Calculate end date for post fire data load
start_date_post = datetime.strftime(((datetime.strptime(fire_date, '%Y-%m-%d'))+timedelta(days=0)), '%Y-%m-%d')
end_date_post = datetime.strftime(((datetime.strptime(fire_date, '%Y-%m-%d'))+timedelta(days=9)), '%Y-%m-%d')

In [45]:
# Print dates
print('start_date_pre:  '+start_date_pre)
print('end_date_pre:    '+end_date_pre)
print('fire_date:       '+fire_date)
print('start_date_post: '+start_date_post)
print('end_date_post:   '+end_date_post)

start_date_pre:  2019-11-21
end_date_pre:    2020-01-26
fire_date:       2020-02-20
start_date_post: 2020-02-20
end_date_post:   2020-02-29


In [47]:
def dNBR_processing(coordinates):
    
    # Load all data in baseline period available from s2a/b_ard_granule datasets
    baseline_ard = load_ard(dc=dc,
                   products=['s2a_ard_granule', 's2b_ard_granule'],
                   x = (coordinates.x - 0.1, coordinates.x + 0.1),
                   y = (coordinates.y - 0.1, coordinates.y + 0.1),
                   time=(start_date_pre, end_date_pre),
                   measurements=['nbart_nir_1',
                                 'nbart_swir_3'],
                   min_gooddata=0.1,
                   output_crs='EPSG:32756',
                   resolution=(-10,10),
                   group_by='solar_day')

    # Remove milliseconds 
    baseline_ard['time'] = baseline_ard.indexes['time'].normalize()
    
    baseline_ard = calculate_indices(baseline_ard,
                                     index='NBR', 
                                     collection='ga_s2_1', 
                                     drop=False)

    # Compute median using all observations in the dataset along the time axis
    baseline_image = baseline_ard.median(dim='time')

    # Delete baseline_combined
    del baseline_ard

    # Select NBR
    baseline_NBR = baseline_image.NBR

    del baseline_image

    # Load all data in post-fire period available from s2a/b_ard_granule datasets
    post_col = load_ard(dc=dc,
               products=['s2a_ard_granule', 's2b_ard_granule'],
               x = (coordinates.x - 0.1, coordinates.x + 0.1),
               y = (coordinates.y - 0.1, coordinates.y + 0.1),
               time=(start_date_post, end_date_post),
               measurements=['nbart_nir_1',
                             'nbart_swir_3'],
               min_gooddata=0.1,
               output_crs='EPSG:32756',
               resolution=(-10,10),
               group_by='solar_day')

    # Calculate NBR on all post-fire images
    post_combined = calculate_indices(post_col, index='NBR', collection='ga_s2_1', drop=False)

    del post_col

    # Calculate the median post-fire image
    post_image = post_combined.median(dim='time')

    del post_combined

    # Select NBR
    post_NBR = post_image.NBR

    del post_image

    # Calculate delta
    delta_NBR = baseline_NBR - post_NBR

    del baseline_NBR
    del post_NBR
    
    x = np.round_(coordinates.x, decimals=4)
    y = np.round_(coordinates.y, decimals=4)

    # Turn dNBR into a x-array dataset for export to GeoTIFF
    dnbr_dataset = delta_NBR.to_dataset(name='burnt_area')
    write_geotiff(f'./geotiffs/{x}_{y}_dNBR.tif', dnbr_dataset)
        
    del delta_NBR
    del dnbr_dataset

In [48]:
# Iterate through all shapely points to generate a dNBR geotiff.
for i in coords:
    dNBR_processing(i)

Loading s2a_ard_granule data
    Filtering to 12 out of 14 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 10 out of 12 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 22 observations 
Loading s2a_ard_granule data
    Filtering to 1 out of 1 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 2 out of 2 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 3 observations 


/g/data/v10/public/modules/dea/20200605/lib/python3.6/site-packages/datacube/helpers.py:34: DeprecationWarning: Function datacube.helpers.write_geotiff is deprecated,
please use datacube.utils.cog.write_cog instead
  category=DeprecationWarning)


Loading s2a_ard_granule data
    Filtering to 10 out of 14 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 11 out of 12 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 21 observations 
Loading s2a_ard_granule data
    Filtering to 1 out of 1 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 2 out of 2 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 3 observations 


/g/data/v10/public/modules/dea/20200605/lib/python3.6/site-packages/datacube/helpers.py:34: DeprecationWarning: Function datacube.helpers.write_geotiff is deprecated,
please use datacube.utils.cog.write_cog instead
  category=DeprecationWarning)


Loading s2a_ard_granule data
    Filtering to 11 out of 14 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 10 out of 12 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 21 observations 
Loading s2a_ard_granule data
    Filtering to 1 out of 1 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 2 out of 2 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 3 observations 


...

/g/data/v10/public/modules/dea/20200605/lib/python3.6/site-packages/datacube/helpers.py:34: DeprecationWarning: Function datacube.helpers.write_geotiff is deprecated,
please use datacube.utils.cog.write_cog instead
  category=DeprecationWarning)


Loading s2a_ard_granule data
    Filtering to 5 out of 7 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 5 out of 6 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 10 observations 
Loading s2a_ard_granule data
    No data for s2a_ard_granule
Loading s2b_ard_granule data
    Filtering to 1 out of 1 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 1 observations 


/g/data/v10/public/modules/dea/20200605/lib/python3.6/site-packages/datacube/helpers.py:34: DeprecationWarning: Function datacube.helpers.write_geotiff is deprecated,
please use datacube.utils.cog.write_cog instead
  category=DeprecationWarning)


Loading s2a_ard_granule data
    Filtering to 7 out of 11 observations
    Applying pixel quality mask
Loading s2b_ard_granule data
    Filtering to 4 out of 9 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 11 observations 
Loading s2a_ard_granule data
    No data for s2a_ard_granule
Loading s2b_ard_granule data
    Filtering to 1 out of 1 observations
    Applying pixel quality mask
Combining and sorting data
Masking out invalid values
    Returning 1 observations 


/g/data/v10/public/modules/dea/20200605/lib/python3.6/site-packages/datacube/helpers.py:34: DeprecationWarning: Function datacube.helpers.write_geotiff is deprecated,
please use datacube.utils.cog.write_cog instead
  category=DeprecationWarning)
